<img src="https://cdn.comet.ml/img/notebook_logo.png">

# Anomalib + Comet


## ⚙ **Step 0:** Setup and Installation
Clone the Anomalib project into your environment and install the necessary dependencies.

In [ ]:
!pip install comet_ml --q
import comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.8/501.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/openvinotoolkit/anomalib.git
%cd anomalib
!pip install . --q

Cloning into 'anomalib'...
remote: Enumerating objects: 28365, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 28365 (delta 98), reused 194 (delta 57), pack-reused 28073
Receiving objects: 100% (28365/28365), 1.50 GiB | 31.26 MiB/s, done.
Resolving deltas: 100% (15667/15667), done.
/content/anomalib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.

## ⚙ **Step 1:** Configure Comet Credentials

If you don't already have a Comet account, you can sign up for free [here](https://www.comet.com/signup?utm_source=colab&utm_medium=referral&utm_campaign=AMS_US_EN_AWA_Online_Anomalib_Comet_Integration). Make sure to grab your API key from your account settings and configure your Comet credentials in [any of several ways](https://www.comet.com/docs/v2/guides/tracking-ml-training/configuring-comet/).

For interactive environments like Colab or Jupyter notebooks, the easiest way to do so is often by using the following command. Note that the project parameter is optional, and will default to the name of the dataset used.


In [ ]:
comet_ml.init(api_key="<YOUR-COMET-API-KEY-HERE>")

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


## ⚙ **Step 2:** Modify the Anomalib config file
Next, we'll need to modify our Anomalib config file to enable logging. The easiest way to do this is to open `anomalib/anomalib/models/<model-of-your-choice>/config.yaml` and adjust the relevant parameters accordingly. For our purposes, we only need to adjust the following lines:

```
visualization:
  show_images: true
  save_images: true
  log_images: true
  mode: full # options: ["full", "simple"]

 logging:
  logger: comet
  log_graph: true
```

If you'd rather keep the default config template and create a second, customized version, we've written a `yaml` file below to use with the FastFlow model on the MVTec dataset. Feel free to use and modify this code for your particular use case, but be aware that **each model has a different config file structure.**

In [ ]:
import yaml

config_file = {
    "dataset": {
        "name": "mvtec",
        "format": "mvtec",
        "path": "./data/MVTec",
        "task": "segmentation",
        "category": "bottle",
        "image_size": 256,
        "train_batch_size": 32,
        "test_batch_size": 32,
        "num_workers": 8,
        "normalization": "imagenet",
        "test_split_mode": "from_dir",
        "test_split_ratio": 0.2,
        "val_split_mode": "same_as_test",
        "val_split_ratio": 0.5,
        "transform_config": {"train": None, "val": None},
        "create_validation_set": False,
        "tiling": {
            "apply": False,
            "tile_size": None,
            "stride": None,
            "remove_border_count": 0,
            "use_random_tiling": False,
            "random_til_count": 16,
        },
    },
    "model": {
        "name": "fastflow",
        "backbone": "resnet18",
        "pre_trained": True,
        "flow_steps": 8,
        "hidden_ratio": 1.0,
        "conv3x3_only": True,
        "lr": 0.001,
        "weight_decay": 0.00001,
        "early_stopping": {
            "patience": 3,
            "metric": "pixel_AUROC",
            "mode": "max",
        },
        "normalization_method": "min_max",
    },
    "metrics": {
        "image": ["F1Score", "AUROC"],
        "pixel": ["F1Score", "AUROC"],
        "threshold": {"image_default": 0, "pixel_default": 0, "adaptive": True},
    },
    "visualization": {
        "show_images": False,
        "save_images": True,
        "log_images": True,
        "image_save_path": None,
        "mode": "full",
    },
    "project": {"seed": 42, "path": "./results"},
    "logging": {"logger": "comet", "log_graph": True},
    "optimization": {"export_mode": None},
    "trainer": {
        "accelerator": "auto",
        "accumulate_grad_batches": 1,
        "amp_backend": "native",
        "auto_lr_find": False,
        "auto_select_gpus": False,
        "benchmark": False,
        "check_val_every_n_epoch": 1,
        "default_root_dir": None,
        "detect_anomaly": False,
        "deterministic": False,
        "devices": 1,
        "enable_checkpointing": True,
        "enable_model_summary": True,
        "enable_progress_bar": True,
        "fast_dev_run": False,
        "gpus": None,
        "gradient_clip_val": 0,
        "ipus": None,
        "limit_predict_batches": 1.0,
        "limit_test_batches": 1.0,
        "limit_train_batches": 1.0,
        "limit_val_batches": 1.0,
        "log_every_n_steps": 50,
        "max_epochs": 500,
        "max_steps": -1,
        "max_time": None,
        "min_epochs": None,
        "min_steps": None,
        "move_metrics_to_cpu": False,
        "multiple_trainloader_mode": "max_size_cycle",
        "num_nodes": 1,
        "num_processes": None,
        "num_sanity_val_steps": 0,
        "overfit_batches": 0.0,
        "plugins": None,
        "precision": 32,
        "profiler": None,
        "reload_dataloaders_every_n_epochs": 0,
        "replace_sampler_ddp": True,
        "strategy": None,
        "sync_batchnorm": False,
        "tpu_cores": None,
        "track_grad_norm": -1,
        "val_check_interval": 1.0,
    },
}


#%cd anomalib/models/fastflow
with open(r"src/anomalib/models/fastflow/custom_config.yaml", "w") as file:
    documents = yaml.dump(config_file, file)

## ⚙ **Step 3:** Training
By default `!python tools/train.py` runs the PaDiM model on the bottle category from the MVTec AD (CC BY-NC-SA 4.0) dataset.

To use a different algorithm, just switch out the model name in the config file path to another supported algorithm. To use a custom dataset, just update the relevant Anomalib config file accordingly with the path to your dataset.

```
!python tools/train.py --config src/anomalib/models/<specific-model-name>/<config-file>.yaml
```

Alternatively, if we pass a model name to the `--model` flag, the script will automatically find our config file (so long as it is stored in the appropriate folder as `config.yaml`).

```
!python tools/train.py --model <specific-model-name>
```

In [ ]:
!python tools/train.py --config src/anomalib/models/fastflow/custom_config.yaml

To use wandb logger install it using `pip install wandb`
/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:275: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(
Global seed set to 42
2023-04-28 14:56:57,557 - anomalib.data - INFO - Loading the datamodule
2023-04-28 14:56:57,558 - anomalib.data.utils.transform - INFO - No config file has been provided. Using default transforms.
2023-04-28 14:56:57,558 - anomalib.data.utils.transform - INFO - No config file has been provided. Using default transforms.
2023-04-28 14:56:57,558 - anomalib.models - INFO - Loading the model.
2023-04-28 14:56:57,559 - anomalib.models.components.base.anomaly_module - INFO - Initializing FastflowLightning model.
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer.

Now just head on over to the Comet UI to see your results!

[![anomalib-colab-panels.gif](https://s4.gifyu.com/images/anomalib-colab-panels.gif)](https://www.comet.com/site/)



